Import required packages

In [11]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy import distance
import folium
import requests

Import Foursquare credentials from json file created to preserve secrecy and store in global variables CLIENT_ID and CLIENT_SECRET

In [2]:
import json
globals().update(json.loads(open("foursquareCredentials.json", "r") .read()))
VERSION = '20181124' # Foursquare API version

Get longitude and latitude of location

In [3]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Rumbach Sebestyén u., Budapest, 1075 Hungary")
lat = location.latitude
long = location.longitude

Define function degreeLen which for a given lat and long returns the length of a degree of lat and long in km

In [4]:
def degreeLen(lat, long):
    """Returns the lengths of a single degree of latitude and longitude in km for a given latitude and longitude
    """
    # get length of 1 degree at current latidude and logitude
    latDegree = distance.distance((lat - 0.5, long), (lat + 0.5, long)).km
    longDegree = distance.distance((lat, long - 0.5), (lat, long + 0.5)).km
    return latDegree, longDegree
degreeLen(lat, long)

(111.18052869040775, 75.34729533841742)

In [5]:
def grid(lat, long, radius, tick):
    """Returns a DataFrame whose rows are a grid of points within a circle around a location
    :param lat: latitude of centre of grid
    :param long: longitude of centre of grid
    :param radius: radius of grid in km
    :param tick: distance between points in grid in km
    :returns: pandas DataFrame with a row for each point in the grid and 
              columns:
                  'Name': string of latitude and longitude of point separated by a comma
                  'Latitude': latitude of point
                  'Longitude': longitude of point
    """
    # get length of 1 degree at current latidude and logitude
    latDegree, longDegree = degreeLen(lat, long)
    # get length of 1 tick in degrees
    latTick = tick / latDegree
    longTick = tick / longDegree
    # get coordinates of point which is one radius to the South and West of (lat, long)
    bottomLeft = (lat - radius / latDegree, long - radius / longDegree)
    # get tick points along axis
    gridLats = np.arange(2 * radius / tick + 1) * latTick + bottomLeft[0]
    gridLongs = np.arange(2 * radius / tick + 1) * longTick + bottomLeft[1]
    grid = []
    for lat in gridLats:
        for long in gridLongs:
            if distance.distance((location.latitude, location.longitude), (lat, long)).km <= radius:
                grid.append([str(lat) + ', ' + str(long), lat, long])
    grid = pd.DataFrame(grid)
    grid.columns = ['Name', 'Latitude', 'Longitude']
    return grid

Check using Folium

In [6]:
g = grid(lat, long, 2, 0.2)
# create map
map_grid = folium.Map(location=[lat, long], zoom_start=13)

for i in g.index:
    label = folium.Popup(g.at[i, 'Name'], parse_html=True)
    folium.CircleMarker([g.at[i, 'Latitude'], g.at[i, 'Longitude']],
                        radius=2,
                        popup=label,
                        fill=True,
                        fill_opacity=0.5).add_to(map_grid)
       
map_grid

Define function for pulling nearby venues, with optional filter on category Id

In [18]:
def getNearbyVenues(lat, long, radius, categoryId = ''):
    url = ('https://api.foursquare.com/v2/venues/search?' +
            '&client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION) +
            '&intent=browse&ll={},{}&radius={}&categoryId={}'.format(lat, long, radius, categoryId))
    
    venues = requests.get(url).json()['response']['venues']
    venues_list=[]
    
    for v in venues:
        venues_list.append([v['id'], v['name'], v['location']['distance'], v['categories'][0]['name']])
    
    nearby_venues = pd.DataFrame(venues_list)
    nearby_venues.columns = ['id', 'name', 'distance', 'category']
    nearby_venues.set_index('id', inplace=True)
    
    return(nearby_venues)

In [25]:
categoryId = '4bf58dd8d48988d16d941735'
radius = 200
df = getNearbyVenues(lat, long, radius, categoryId)
df

,name,distance,category
id,,,
510c25ede4b0a3ee4308e94a,Központ,188,Café
4c9ce18c542b224bbaade49f,Drum Cafe,100,Café
52054dd511d2778ebd4f1cf3,Blue Bird Cafe,171,Café
514d9e95e4b01bb2769654a3,Solinfo,67,Café
5b9a6c1d1ffe97002c3349c7,Fresh Corner Downtown,187,Café
4c8e63f5a8de224be0182701,Café Vian,182,French Restaurant
54535798498ef0cd9c592e6e,Blue Bird Roastery,137,Café
4b804281f964a520d36130e3,Spinoza Café,87,Restaurant
4b99441cf964a520be6e35e3,Printa Café,111,Art Gallery


In [24]:
def targetFunc(df, radius):
    for i in df.index:
        print(df[i])
targetFunc(g, radius)

KeyError: 0